# Managing conversation history with the ChatGPT model
This sample notebook demonstrates a couple of simple patterns you can use for managing the prompts and conversation history with the ChatGPT model.

In [12]:
# if needed, install and/or upgrade to the latest version of the OpenAI Python library
%pip install openai=0.8

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: 'openai=0.8': Expected end or semicolon (after name and no valid version specifier)
    openai=0.8
          ^
Hint: = is not a valid operator. Did you mean == ?


In [13]:
# import os module & the OpenAI Python library for calling the OpenAI API
# please make sure you have installed required libraries via pip install -r requirements.txt
import os, dotenv
from openai import AzureOpenAI

dotenv.load_dotenv()

False

In [14]:
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://openai-tajamar-1.openai.azure.com/"
os.environ["AZURE_OPENAI_APIKEY"] = "36P4MBmY3mY0E0BaKoV5ofH2euWgRWSW2dMi5kK6TBNHbdgdHFRxJQQJ99BAACYeBjFXJ3w3AAABACOGeiR4"

In [15]:
# Setting up the deployment name
deployment_name = "gpt-4o-mini"

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
# This is the value of the endpoint for your Azure OpenAI resource


# Currently OPENAI API have the following versions available: https://learn.microsoft.com/azure/ai-services/openai/reference
api_version = "2024-07-01-preview"

client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), 
    api_key=os.getenv("AZURE_OPENAI_APIKEY"), 
    api_version=api_version
)

## 1.0 Create the system message for ChatGPT

In [16]:
base_system_message = "You are a helpful assistant."

system_message = f"{base_system_message.strip()}"
print(system_message)

You are a helpful assistant.


## 2.0 Define helper functions



In [17]:
import tiktoken

def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
    encoding = tiktoken.encoding_for_model(model)
    num_tokens = 0
    for message in messages:
        num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":  # if there's a name, the role is omitted
                num_tokens += -1  # role is always required and always 1 token
    num_tokens += 2  # every reply is primed with <im_start>assistant
    return num_tokens

In [18]:
# Defining a function to send the prompt to the ChatGPT model
# More info : https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions
def send_message(messages, model_name, max_response_tokens=500):
    response = client.chat.completions.create(
        model=model_name,
        messages=messages
    )
    return response.choices[0].message.content

# Defining a function to print out the conversation in a readable format
def print_conversation(messages):
    for message in messages:
        print(f"[{message['role'].upper()}]")
        print(message['content'])
        print()

## 3.0 Start the conversation

In [19]:
# This is the first user message that will be sent to the model. Feel free to update this.
user_message = "I want to write a blog post about the impact of AI on the future of work."

In [20]:
# Create the list of messages. role can be either "user" or "assistant" 
messages= messages = [
    {"role": "system", "content": "You are a helpful assistant."},  
    {"role": "user", "content": "Can you explain how to use Azure OpenAI ChatGPT API?"}  
]

In [21]:
token_count = num_tokens_from_messages(messages)
print(f"Token count: {token_count}")

Token count: 32


In [22]:
max_response_tokens = 100

response = send_message(messages, deployment_name, max_response_tokens)
messages.append({"role": "assistant", "content": response})

print_conversation(messages)

[SYSTEM]
You are a helpful assistant.

[USER]
Can you explain how to use Azure OpenAI ChatGPT API?

[ASSISTANT]
Certainly! To use the Azure OpenAI ChatGPT API, you'll need to follow several steps to set up your Azure account, create an OpenAI resource, and then make API calls. Here’s a step-by-step guide:

### Step 1: Set Up Your Azure Account

1. **Create an Azure Account**: 
   - Go to the Azure website (https://azure.microsoft.com) and sign up for an account. If you already have an account, log in.

2. **Subscription**: 
   - You might need an appropriate Azure subscription that includes access to the OpenAI service.

### Step 2: Create an OpenAI Resource

1. **Navigate to Azure Portal**: 
   - Go to the Azure portal (https://portal.azure.com).

2. **Create a Resource**:
   - Click on “Create a resource” from the top left corner.

3. **Search for OpenAI**:
   - In the search bar, type "OpenAI" and select "OpenAI Service".

4. **Fill in the Details**:
   - Provide the required inform

## 4.0 Continue the conversation

When working with the ChatGPT model, it's your responsibity to make sure you stay within the token limits of the model. The model can handle a maximum of 4096 tokens, and this includes the number of tokens in the prompt as well as the `max_tokens` you're requesting from the model. If you exceed these limits, the model will return an error.

You should also consider the trade-off between maintaining more of the conversation history and the cost/latency that you'll incur by including those tokens in the prompt. Shorter prompts are cheaper and faster. The amount of the previous conversation you include also makes a difference in how the model responds.

In this notebook, we'll show two strategies for managing the conversation history when working with the ChatGPT model.
- Option 1: Keep the conversation within a given token limit
- Option 2: Keep the conversation within a given number of turns

### Option 1: Keep the conversation within a given token limit

`overall_max_tokens` is the maximum number of tokens that you want to include in the prompt. Th maximum number this can be set to is 4096 but you can also consider reducing this number to reduce the cost and latency of the request.

In [23]:
overall_max_tokens = 1096
prompt_max_tokens = overall_max_tokens - max_response_tokens

You can continue the conversation below by editing the user_message and running the cell as many times as you would like.

In [24]:
user_message = "The target audience for the blog post should be business leaders working in the tech industry."
#user_message = "Let's talk about generative AI and keep the tone informational but also friendly."
#user_message = "Show me a few more examples"
messages.append({"role": "user", "content": user_message})

token_count = num_tokens_from_messages(messages)
print(f"Token count: {token_count}")

# remove first message while over the token limit
while token_count > prompt_max_tokens:
    messages.pop(0)
    token_count = num_tokens_from_messages(messages)

response = client.chat.completions.create(
        model=deployment_name,
        messages=messages
    )

#Append assistance response 
messages.append({"role": "assistant", "content": response.choices[0].message.content})

print_conversation(messages)

Token count: 930
[SYSTEM]
You are a helpful assistant.

[USER]
Can you explain how to use Azure OpenAI ChatGPT API?

[ASSISTANT]
Certainly! To use the Azure OpenAI ChatGPT API, you'll need to follow several steps to set up your Azure account, create an OpenAI resource, and then make API calls. Here’s a step-by-step guide:

### Step 1: Set Up Your Azure Account

1. **Create an Azure Account**: 
   - Go to the Azure website (https://azure.microsoft.com) and sign up for an account. If you already have an account, log in.

2. **Subscription**: 
   - You might need an appropriate Azure subscription that includes access to the OpenAI service.

### Step 2: Create an OpenAI Resource

1. **Navigate to Azure Portal**: 
   - Go to the Azure portal (https://portal.azure.com).

2. **Create a Resource**:
   - Click on “Create a resource” from the top left corner.

3. **Search for OpenAI**:
   - In the search bar, type "OpenAI" and select "OpenAI Service".

4. **Fill in the Details**:
   - Provide th

### Option 2: Keep the conversation within a given number of turns

In [25]:
max_messages = 10

overall_max_tokens = 4096
prompt_max_tokens = overall_max_tokens - max_response_tokens

You can continue the conversation below by editing the user_message and running the cell as many times as you would like.

In [26]:
user_message = "Make the post about generative AI aimed at business leaders who have some knowledge of the technology."
messages.append({"role": "user", "content": user_message})

while len(messages) > max_messages:
    messages.pop(0)

token_count = num_tokens_from_messages(messages)

while token_count > prompt_max_tokens:
    # remove first message from messages
    messages.pop(0)
    token_count = num_tokens_from_messages(messages)

response = client.chat.completions.create(
        model=deployment_name,
        messages=messages
    )
    
#Append assistant response
messages.append({"role": "assistant", "content": response.choices[0].message.content})
print_conversation(messages)


[SYSTEM]
You are a helpful assistant.

[USER]
Can you explain how to use Azure OpenAI ChatGPT API?

[ASSISTANT]
Certainly! To use the Azure OpenAI ChatGPT API, you'll need to follow several steps to set up your Azure account, create an OpenAI resource, and then make API calls. Here’s a step-by-step guide:

### Step 1: Set Up Your Azure Account

1. **Create an Azure Account**: 
   - Go to the Azure website (https://azure.microsoft.com) and sign up for an account. If you already have an account, log in.

2. **Subscription**: 
   - You might need an appropriate Azure subscription that includes access to the OpenAI service.

### Step 2: Create an OpenAI Resource

1. **Navigate to Azure Portal**: 
   - Go to the Azure portal (https://portal.azure.com).

2. **Create a Resource**:
   - Click on “Create a resource” from the top left corner.

3. **Search for OpenAI**:
   - In the search bar, type "OpenAI" and select "OpenAI Service".

4. **Fill in the Details**:
   - Provide the required inform

## Ejercicio 1: Modificar el Mensaje del Sistema
Enunciado:
Crea un mensaje del sistema que defina a ChatGPT como un "analista financiero experto". Imprime el mensaje del sistema resultante y utilízalo en un mensaje inicial del usuario que pregunte:
"¿Cuáles son las tendencias actuales en el mercado de acciones tecnológicas?".


In [29]:

# Mensaje del sistema
base_system_message = "You are a financial analyst expert."
system_message = f"{base_system_message.strip()}"
print(system_message)

# Crear lista de mensajes
user_message = "What are the current trends in the tech stock market?"
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": "What are the current trends in the tech stock market?"}
]

# Enviar mensaje y recibir respuesta
response = client.chat.completions.create(
    model= deployment_name,
    messages= messages,
    max_tokens= 100,  
    temperature= 0.7
)


messages.append({"role": "assistant", "content": response.choices[0].message.content})
print_conversation(messages)


You are a financial analyst expert.
[SYSTEM]
You are a financial analyst expert.

[USER]
What are the current trends in the tech stock market?

[ASSISTANT]
As of October 2023, several key trends have been shaping the tech stock market:

1. **Artificial Intelligence (AI) Boom**: The surge in AI technologies has driven significant interest and investment in companies involved in AI development, including software and hardware providers. Major players like NVIDIA and Microsoft have seen substantial gains due to their focus on AI capabilities.

2. **Cloud Computing Growth**: The shift to cloud services continues to accelerate, with companies like Amazon (AWS), Microsoft (Azure), and



## Ejercicio 2: Contar Tokens en una Conversación Extendida
Enunciado:
Dado un conjunto inicial de mensajes, agrega tres mensajes adicionales:

"Explain the impact of inflation on stock prices."
"What are the top stocks to watch in 2025?"
"Summarize the latest market news in 50 words."
Calcula y muestra el número total de tokens tras añadir cada mensaje.


In [30]:

# Mensajes iniciales
messages = [
    {"role": "system", "content": "You are a helpful assistant specialized in financial topics."},
    {"role": "user", "content": "What are the current trends in the tech stock market?"}
]

# Mensajes adicionales
additional_messages = [
    "Explain the impact of inflation on stock prices.",
    "What are the top stocks to watch in 2025?",
    "Summarize the latest market news in 50 words."
]

# Agregar mensajes y calcular tokens
for msg in additional_messages:
    messages.append({"role": "user", "content": msg})
    token_count = num_tokens_from_messages(messages)
    print(f"Token count after adding '{msg}': {token_count}")


Token count after adding 'Explain the impact of inflation on stock prices.': 48
Token count after adding 'What are the top stocks to watch in 2025?': 65
Token count after adding 'Summarize the latest market news in 50 words.': 82


## Ejercicio 3: Personalizar el Mensaje del Sistema Según el Usuario
Enunciado:
Escribe un programa que permita al usuario definir el rol de ChatGPT mediante una entrada de texto (input). Una vez definido, responde al mensaje:
"What are your key skills as an assistant?"
Usa el mensaje del sistema definido por el usuario.


In [ ]:

# Entrada del usuario para definir el mensaje del sistema
user_defined_role = input("Define the role of the assistant: ")
system_message = f"You are {user_defined_role.strip()}."

# Crear lista de mensajes
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": "What are your key skills as an assistant?"}
]

# Enviar mensaje y recibir respuesta
response = client.chat.completions.create(
    model= deployment_name,
    messages= messages,
    max_tokens= 100,  
    temperature= 0.7
)

messages.append({"role": "assistant", "content": response})

# Mostrar la conversación
print_conversation(messages)


## Ejercicio 4: Crear un Generador de Respuestas Creativo
Enunciado:
Diseña un programa que permita configurar la creatividad del modelo (temperatura) a través de una entrada del usuario. Una vez configurado, responde a la pregunta:
"Can you suggest an out-of-the-box idea for a startup?"



In [1]:
# Entrada del usuario para definir la creatividad
temperature = float(input("Set the creativity level (0.0 - 1.0): "))
if not (0.0 <= temperature <= 1.0):
    raise ValueError("Creativity level must be between 0.0 and 1.0.")

# Crear lista de mensajes
system_message = "You are a highly creative assistant."
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": "Can you suggest an out-of-the-box idea for a startup?"}
]

# Enviar mensaje con creatividad ajustada
response = client.chat.completions.create(
    model=deployment_name,
    messages=messages,
    max_tokens=100,  
    temperature=0.7
)

# Mostrar la respuesta
print(response.choices[0].message.content)

ValueError: could not convert string to float: ''

## Ejercicio 5: Cambiar el Tono del Mensaje
Enunciado:
Configura el mensaje del sistema para que ChatGPT responda en un tono humorístico y escribe una respuesta al mensaje:
"Explain quantum computing in a funny way."


In [60]:
# Mensaje del sistema humorístico
system_message = "You are a humorous assistant."
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": "Explain quantum computing in a funny way."}
]

# Enviar mensaje y recibir respuesta
response = client.chat.completions.create(
    model=deployment_name,
    messages=messages,
    max_tokens=100,  
    temperature=0.7
)

#append assistant response
messages.append({"role": "assistant", "content": response.choices[0].message.content})
print_conversation(messages)


[SYSTEM]
You are a humorous assistant.

[USER]
Explain quantum computing in a funny way.

[ASSISTANT]
Sure! Imagine quantum computing as a party where all the computers are invited, but instead of just hanging out like regular computers, they decide to throw a wild dance-off.

In the classical computing corner, you have traditional computers. They’re like that one friend who insists on following the rules: "I can only be in one place at a time, and I can only dance one way." They can do calculations, but they do them step by step, like a very organized line dance.

Now, over in the quantum corner, things get a bit wacky. These computers are like that friend who shows up in a sequined outfit and starts doing the cha-cha while simultaneously breakdancing and moonwalking. They can be in multiple states at once, thanks to something called superposition. So while the classical computers are busy doing the Macarena, quantum computers are out there doing the entire dance floor at once!

And t

## Ejercicio 6: Resumir Mensajes Previos
Enunciado:
Dado un historial de conversación de 5 mensajes, crea una función que resuma el contenido de los mensajes del usuario en una sola oración.



In [63]:
# Historial de mensajes
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Tell me about AI."},
    {"role": "user", "content": "How does machine learning work?"},
    {"role": "user", "content": "Can you explain neural networks?"},
    {"role": "user", "content": "What are the applications of deep learning?"}
]

# Resumir mensajes del usuario
def summarize_user_messages(messages):
    user_messages = [msg['content'] for msg in messages if msg['role'] == 'user']
    summary_prompt = "Summarize the following messages into one sentence: " + " ".join(user_messages)
    summary = send_message([{"role": "system", "content": "You are a helpful assistant."},
                            {"role": "user", "content": summary_prompt}], deployment_name)
    return summary

# Obtener y mostrar el resumen
summary = summarize_user_messages(messages)
print(f"User messages summary: {summary}")


User messages summary: AI encompasses various techniques, including machine learning, which involves training algorithms on data, neural networks that simulate brain-like structures for processing information, and deep learning applications that range from image and speech recognition to natural language processing.


## Ejercicio 7: Configuración de Temperatura y Tono
Enunciado:
Escribe un programa que cambie la configuración de temperatura a 0.9 para obtener respuestas más creativas y responde al mensaje:
"Suggest a unique title for a blog about AI in education."



In [66]:

# Mensajes iniciales
messages = [
    {"role": "system", "content": "You are a creative assistant."},
    {"role": "user", "content": "Suggest a unique title for a blog about AI in education."}
]

# Ajustar temperatura y enviar mensaje
response = client.chat.completions.create(
    model=deployment_name,
    messages=messages,
    max_tokens=100,  
    temperature=0.7
)

# Mostrar la respuesta
print(response.choices[0].message.content)


"Learning Reimagined: Navigating the Future of Education with AI"
